<a href="https://colab.research.google.com/github/nse4real/myrepo/blob/master/stylometry_grouping_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GROUPING OF 2 STYLOMETRY

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nltk
nltk.download('stopwords')
#nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
#IMPORT NEEDED LIBRARIES
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import re, string, unicodedata, math
import collections as coll
from nltk.corpus import words as nltk_words
# Import other models needed for Classification models
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from keras.models import Sequential
#from keras.layers.core import Dense
from sklearn.cluster import KMeans
import tensorflow as tf
import keras
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from imutils import paths
import argparse
import random
import pickle
import cv2
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.utils.class_weight import compute_class_weight
import csv
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from tabulate import tabulate
from matplotlib import style
style.use("ggplot")

In [4]:
path= '/content/drive/MyDrive/Colab Notebooks/'
filepath= '/content/drive/MyDrive/Colab Notebooks/dataset.csv'
print(filepath)

/content/drive/MyDrive/Colab Notebooks/dataset.csv


In [5]:
# read the csv file
df = pd.read_csv(filepath)

# print number df rows and columns
print(df.shape)
  
# print the content
print('Content:')
#display(df)

#print the column names
print(df.columns)
print()

df.head()

(4560, 18)
Content:
Index(['input.access_token', 'input.exclude', 'input.tweet_fields',
       'input.user_id', 'input.pagination_token',
       'data.public_metrics.retweet_count', 'data.public_metrics.reply_count',
       'data.public_metrics.like_count', 'data.public_metrics.quote_count',
       'data.source', 'data.text', 'data.id', 'data.created_at',
       'meta.oldest_id', 'meta.newest_id', 'meta.result_count',
       'meta.next_token', 'meta.previous_token'],
      dtype='object')



,input.access_token,input.exclude,input.tweet_fields,input.user_id,input.pagination_token,data.public_metrics.retweet_count,data.public_metrics.reply_count,data.public_metrics.like_count,data.public_metrics.quote_count,data.source,data.text,data.id,data.created_at,meta.oldest_id,meta.newest_id,meta.result_count,meta.next_token,meta.previous_token
0,AAAAAAAAAAAAAAAAAAAAAAyUYgEAAAAAYT3ZR%2Bw6cIB8...,"retweets,replies","public_metrics,created_at,text,source",27699224,NaN,0,1,15,0,Twitter for iPhone,"FYI for other authors…this raised $1,460 which...",1498672192766328838,2022-03-01T14:51:09.000Z,1396442583505805316,1498672192766328838,100,7140dibdnow9c7btw3w4sioouy7v4owjrs89d2hqa0yly,NaN
1,AAAAAAAAAAAAAAAAAAAAAAyUYgEAAAAAYT3ZR%2Bw6cIB8...,"retweets,replies","public_metrics,created_at,text,source",27699224,NaN,2,4,66,0,Twitter for iPhone,"Twenty years from now, this is gonna read like...",1498655705351479296,2022-03-01T13:45:38.000Z,1396442583505805316,1498672192766328838,100,7140dibdnow9c7btw3w4sioouy7v4owjrs89d2hqa0yly,NaN
2,AAAAAAAAAAAAAAAAAAAAAAyUYgEAAAAAYT3ZR%2Bw6cIB8...,"retweets,replies","public_metrics,created_at,text,source",27699224,NaN,7,3,67,1,Twitter for iPhone,If you donate $20 or more to @WCKitchen and DM...,1498048242336120832,2022-02-27T21:31:48.000Z,1396442583505805316,1498672192766328838,100,7140dibdnow9c7btw3w4sioouy7v4owjrs89d2hqa0yly,NaN
3,AAAAAAAAAAAAAAAAAAAAAAyUYgEAAAAAYT3ZR%2Bw6cIB8...,"retweets,replies","public_metrics,created_at,text,source",27699224,NaN,0,1,23,1,Twitter Web App,Our long international nightmare is just begin...,1496922333512351744,2022-02-24T18:57:50.000Z,1396442583505805316,1498672192766328838,100,7140dibdnow9c7btw3w4sioouy7v4owjrs89d2hqa0yly,NaN
4,AAAAAAAAAAAAAAAAAAAAAAyUYgEAAAAAYT3ZR%2Bw6cIB8...,"retweets,replies","public_metrics,created_at,text,source",27699224,NaN,0,0,20,0,Twitter for iPhone,Nostalgia. https://t.co/oh9hsNGL0U,1495551477057732609,2022-02-21T00:10:32.000Z,1396442583505805316,1498672192766328838,100,7140dibdnow9c7btw3w4sioouy7v4owjrs89d2hqa0yly,NaN


In [6]:
#DROP UNNECCESSARY COLUMNS
columns_to_drop = ['input.access_token', 'input.exclude',
       'input.tweet_fields', 'input.pagination_token', 'meta.oldest_id',
       'meta.newest_id', 'meta.result_count', 'meta.next_token',
       'meta.previous_token', 'data.public_metrics.retweet_count',
       'data.public_metrics.reply_count', 'data.public_metrics.like_count',
       'data.public_metrics.quote_count', 'data.source',
       'data.id', 'data.created_at']
df=df.drop(columns=columns_to_drop)
print(df.columns)
df.head()

Index(['input.user_id', 'data.text'], dtype='object')


,input.user_id,data.text
0,27699224,"FYI for other authors…this raised $1,460 which..."
1,27699224,"Twenty years from now, this is gonna read like..."
2,27699224,If you donate $20 or more to @WCKitchen and DM...
3,27699224,Our long international nightmare is just begin...
4,27699224,Nostalgia. https://t.co/oh9hsNGL0U


In [7]:
#RENAME THE COLUMNS NAMES TO SHORTER NAMES
new_columns={'input.user_id':'user_id', 'data.text':'tweet'}
df=df.rename(columns=new_columns)
# print(df.head())
print(df.columns)

Index(['user_id', 'tweet'], dtype='object')


In [8]:
#ADD NEW COLUMN CALLED USERNAME
authors = {
  426044697: "@Shteyngart",
  18903971: "@harlancoben ",
  27699224: "@askanyone",
  17790667: "@megcabot",
  5520952: "@paulocoelho",
  83876527: "@tejucole"
}
#print(authors)

df['username'] = "" #Add an empty column
#df.info()

for id, username in authors.items():
  #print(df[df.user_id==id]);
  df.loc[df.user_id==id, ['username']] = username

#print userid and username columns
print(df[['user_id','username']].head())
print(df.shape)

    user_id    username
0  27699224  @askanyone
1  27699224  @askanyone
2  27699224  @askanyone
3  27699224  @askanyone
4  27699224  @askanyone
(4560, 3)


### Data Cleaning

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   4560 non-null   int64 
 1   tweet     4560 non-null   object
 2   username  4560 non-null   object
dtypes: int64(1), object(2)
memory usage: 107.0+ KB


In [10]:
## preprocessing functions
def remove_URL(tweet):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", tweet)

def remove_numbers(tweet):
  """Remove words with numbers"""
  return re.sub("[0-9]","",tweet)   

def remove_non_ascii(tweet):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', tweet.encode('ascii', 'ignore').decode('utf-8', 'ignore'))

In [11]:
# Use apply function to apply preprocessing
df["tweet"] = df["tweet"].apply(lambda t: remove_URL(t)) #remove urls
df["tweet"] = df["tweet"].apply(lambda t: remove_numbers(t))
df["tweet"] = df["tweet"].apply(lambda t: remove_non_ascii(t))
df["tweet"] = df["tweet"].apply(lambda t: t.strip())
#print(df.shape)

In [12]:
## C0nvert username to categorical type
df["username"] = df["username"].astype('category')
df["username_code"] = df["username"].cat.codes
ucat = df["username"].values
print(ucat.categories)
print(np.unique(ucat.codes))

Index(['@Shteyngart', '@askanyone', '@harlancoben ', '@megcabot',
       '@paulocoelho', '@tejucole'],
      dtype='object')
[0 1 2 3 4 5]


In [13]:
# CHECK FOR EMPTY TWEET
df_empty_tweet = df[df['tweet'].str.len()<1]
print("Number of empty tweet: "+str(df_empty_tweet.shape[0]))
df_empty_tweet.shape

Number of empty tweet: 159


(159, 4)

In [14]:
# REMOVE EMPTY TWEETS
df2 = df[df['tweet'].str.len()>0]
print("Cleaned dataset size: {0}",df2.shape[0])
print(df2.shape)
df2.head()

Cleaned dataset size: {0} 4401
(4401, 4)


,user_id,tweet,username,username_code
0,27699224,"FYI for other authorsthis raised $, which is a...",@askanyone,1
1,27699224,"Twenty years from now, this is gonna read like...",@askanyone,1
2,27699224,If you donate $ or more to @WCKitchen and DM m...,@askanyone,1
3,27699224,Our long international nightmare is just begin...,@askanyone,1
4,27699224,Nostalgia.,@askanyone,1


### Perform grouping

In [15]:
#Get the columns in the dataframe,#create a new dataframe
group_2_df = pd.DataFrame(columns=df2.columns)
total_rows = 0;
#for each author,
for username_code in range(6):
  author_tweets = df2.loc[df2.username_code==username_code]
  #Get the total number of rows in the dataset
  rows =author_tweets.shape[0]
  tweet_index = author_tweets.index
  print(f"The number of rows for author: {username_code} is {rows}")
  #loop through the df and merge two consercutive tweets
  i = 0
  while i < (rows-1):
    #add row to the new dataframe
    row = author_tweets.loc[tweet_index[i]]
    tweet = author_tweets.loc[tweet_index[i],["tweet"]] + " " + author_tweets.loc[tweet_index[i+1],["tweet"]]
    #row["tweet"] = tweet
    group_2_df.loc[total_rows] = row
    group_2_df.loc[total_rows,["tweet"]] = tweet
    i+=2
    total_rows +=1

print("The grouped df shape is: ", group_2_df.shape)

group_2_df["username"] = group_2_df["username"].astype('category')
group_2_df["username_code"] = group_2_df["username"].cat.codes
ucat = group_2_df["username"].values
print(ucat.categories)
print(np.unique(ucat.codes))
group_2_df.head()

The number of rows for author: 0 is 840
The number of rows for author: 1 is 761
The number of rows for author: 2 is 754
The number of rows for author: 3 is 697
The number of rows for author: 4 is 651
The number of rows for author: 5 is 698
The grouped df shape is:  (2199, 4)
Index(['@Shteyngart', '@askanyone', '@harlancoben ', '@megcabot',
       '@paulocoelho', '@tejucole'],
      dtype='object')
[0 1 2 3 4 5]


,user_id,tweet,username,username_code
0,426044697,A brilliant look at Ukraines history through a...,@Shteyngart,0
1,426044697,There might just be a God. I was wondering whe...,@Shteyngart,0
2,426044697,Question: How can we remove Russia from its pe...,@Shteyngart,0
3,426044697,"My math is terrible, but if troops were kille...",@Shteyngart,0
4,426044697,To the barricades! At yesterdays PEN vigil for...,@Shteyngart,0


### Feature Extraction

In [16]:
#Dictionary words
dictionary = nltk_words.words();
english_words = set(word.strip().lower() for word in dictionary)

# characters or numbers appearing 3 or more consecutive times in a string  
# Compile the ReGex //"\\b([a-zA-Z0-9])\\1\\1+\\b"
repeating_re = re.compile(r"(.)\1{2,}")
tokenizer = nltk.RegexpTokenizer(r"\w+")

In [17]:
### Lexical and syntactic Feature Extraction
def lexical_feature_extration(tweet):
  tk_no_puntuation = tokenizer.tokenize(tweet)
  tk_punctuation = tweet.split(" ")
  
  # number of words per tweet
  num_words = len(tk_punctuation)

  # number of sentences
  sentences = nltk.sent_tokenize(tweet)
  num_sentences = len(sentences)

  #number of begining words capitalized, #number of punctuation per sentence, #number of uppercase letters
  num__beginning_sent_capital = 0
  num_punct_sent = 0
  num_uppers_sent = 0
  for s in sentences:
    if s[0].isupper():
      num__beginning_sent_capital +=1
    for char in s:
      if char in string.punctuation:
          num_punct_sent +=1
      if char.isupper():
        num_uppers_sent +=1

  # number of words for per sentences
  num_sent_words = [len(s.split(" ")) for s in sentences]
  word_per_sent = sum(num_sent_words)/num_sentences

  #Frequency of dictionary words
  freq_dict_words = sum([1 if word.lower() in english_words else 0 for word in tk_no_puntuation])

  #Frequency of word extensions
  freq_words_ex = sum([1 if re.search(repeating_re, word) else 0 for word in tk_no_puntuation])

  #lexical diversity
  #print(tk_punctuation)
  lex_diversity = len(tk_no_puntuation)/len(set(tk_no_puntuation)) if len(tk_no_puntuation)>0 else 0

  return pd.Series([num_words,num_sentences,word_per_sent,freq_dict_words,freq_words_ex,lex_diversity,
                    num__beginning_sent_capital,num_punct_sent,num_uppers_sent])

In [18]:
## Apply lexical feature extraction
group_2_df[['num_words','num_sentences','word_per_sent','freq_dict_words','freq_words_ex','lex_diversity','num__beginning_sent_capital','num_punct_sent','num_uppers_sent']] = group_2_df['tweet'].apply(lexical_feature_extration)

#add columns mimicry_by_length and mimicry_by_word to list
group_2_df['mimicry_by_length'] = np.nan
group_2_df['mimicry_by_word'] = np.nan

#mimicry by length code
#get each authors tweet
for username in authors.values():
  #print(df[df.user_id==id]);
  author_data = group_2_df[group_2_df.username==username]
  #sort it
  author_tweets = author_data['tweet']
  #get the list of keys
  tweet_index_list = author_tweets.index
  #get mimicry length and mimicry by word
  mimicry_length = []
  for i in range(len(tweet_index_list)-1):
      currentKey = tweet_index_list[i]
      nextKey = tweet_index_list[i+1]
      #mimicry by length
      group_2_df.loc[currentKey,'mimicry_by_length'] = len(author_tweets[currentKey])/len(author_tweets[nextKey])
      #mimicry by word
      current_tweet_words = set(tokenizer.tokenize(author_tweets[currentKey]))
      next_tweet_words = set(tokenizer.tokenize(author_tweets[nextKey]))
      group_2_df.loc[currentKey,'mimicry_by_word'] = len(current_tweet_words.intersection(next_tweet_words))
      
  group_2_df.loc[tweet_index_list[-1],'mimicry_by_length'] = 0
  group_2_df.loc[tweet_index_list[-1],'mimicry_by_word'] = 0
  
group_2_df.head()

,user_id,tweet,username,username_code,num_words,num_sentences,word_per_sent,freq_dict_words,freq_words_ex,lex_diversity,num__beginning_sent_capital,num_punct_sent,num_uppers_sent,mimicry_by_length,mimicry_by_word
0,426044697,A brilliant look at Ukraines history through a...,@Shteyngart,0,17.0,3.0,5.666667,16.0,0.0,1.000000,3.0,4.0,4.0,0.490385,1.0
1,426044697,There might just be a God. I was wondering whe...,@Shteyngart,0,39.0,3.0,13.333333,36.0,0.0,1.081081,3.0,3.0,8.0,1.118280,4.0
2,426044697,Question: How can we remove Russia from its pe...,@Shteyngart,0,25.0,4.0,6.750000,25.0,0.0,1.000000,4.0,6.0,8.0,1.141104,2.0
3,426044697,"My math is terrible, but if troops were kille...",@Shteyngart,0,32.0,2.0,16.000000,25.0,0.0,1.035714,2.0,4.0,6.0,2.963636,1.0
4,426044697,To the barricades! At yesterdays PEN vigil for...,@Shteyngart,0,9.0,2.0,4.500000,6.0,0.0,1.000000,2.0,2.0,6.0,0.327381,1.0


### Performing classification

In [19]:
def balanceData(balanceType, trainX, trainY):
    # ["None", "SMOTE", "RandomOver", "RandomUnder"]
    if balanceType == "None":
        x_train_res = trainX
        y_train_res = trainY
    if balanceType == "SMOTE":
        sm = SMOTE(random_state=12)
        x_train_res, y_train_res = sm.fit_resample(trainX, trainY)
    if balanceType == "RandomOver":
        sm = RandomOverSampler(random_state=12)
        x_train_res, y_train_res = sm.fit_sample(trainX, trainY)
    if balanceType == "RandomUnder":
        sm = RandomUnderSampler(random_state=12)
        x_train_res, y_train_res = sm.fit_sample(trainX, trainY)

    return x_train_res, y_train_res
        
def initNeuralNetModel(trainX, trainY,numFeatures):
    train_Y = np_utils.to_categorical(trainY)
    inputShape = (numFeatures,)
    class_weight1 = {0: 1.,1: 20.}
    y_ints = [y.argmax() for y in train_Y]
    train_classes = np.unique(y_ints)
    num_classes = len(train_classes)
    model = Sequential()
    model.add(Dense(32, input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(Dense(32))
    model.add(Activation("relu"))
    model.add(Dense(num_classes))
    model.add(Activation(tf.nn.softmax))

    #opt = RMSprop(learning_rate=0.0001, decay=1e-6)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    # opt = RMSprop(0.001)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

def doNeuralNetAnalysis(model, trainX, trainY, testX, testY, numFeatures):
    print("Doing analysis with fully connected neural network")
    train_Y = np_utils.to_categorical(trainY)
    test_Y = np_utils.to_categorical(testY)
    inputShape = (numFeatures,)
    class_weight1 = {0: 1.,1: 20.}
    y_ints = [y.argmax() for y in train_Y]
    train_classes = np.unique(y_ints)
    chanDim = 1
    num_classes = len(train_classes)

    # initialize our initial learning rate and # of epochs to train for
    INIT_LR = 0.01
    EPOCHS = 4

    # compile the model using SGD as our optimizer and categorical
    # cross-entropy loss (you'll want to use binary_crossentropy
    # for 2-class classification)
    print("[INFO] training network...")
    # opt = SGD(lr=INIT_LR)
    # model.compile(loss="categorical_crossentropy", optimizer=opt,
    #              metrics=["accuracy"])
    # model.compile(loss='binary_crossentropy',
    #              optimizer=opt,
    #              metrics=[sensitivity, specificity])
    class_weights = compute_class_weight(class_weight = "balanced",classes=train_classes, y = y_ints)
    class_weights = dict(zip(np.unique(train_classes), class_weights))
    #print(class_weights)

    # train the neural network
    model.fit(trainX, train_Y, validation_data=(testX, test_Y), epochs=EPOCHS, batch_size=32,
                  class_weight=class_weights, verbose=0)

    # evaluate the network
    print("[INFO] evaluating network...")
    #preds = model.predict(testX, verbose=0)
    #predictions = preds.argmax(axis=1)
    #print(classification_report(testY, predictions, target_names=["0", "1"]))
    #print(confusion_matrix(testY, predictions))
    return model

def getClassifierCode(classifierType):
    #["ANN", "SVM", "Logistic Regression", "Decision Tree", "Random Forest", "Naive Bayes"]
    code = ""
    if classifierType == "ANN":
        return "ann"
    if classifierType == "SVM":
        return "svm"
    if classifierType == "Decision Tree":
        return "dt"
    if classifierType == "Random Forest":
        return "rf"
    if classifierType == "Naive Bayes":
        return "NB"
    if classifierType == "Logistic Regression":
      return "LR"   

    return code

def getBalanceCode(balanceType):
    #["None", "SMOTE", "RandomOver", "RandomUnder"]
    code = ""
    if balanceType == "None":
        code = '-'
    if balanceType == "SMOTE":
        code = 's'
    if balanceType == "RandomOver":
        code = 'ro'
    if balanceType == "RandomUnder":
        code = 'ru'
    return code

In [20]:
print(group_2_df.columns)
# We did not consider the non-stylometric feature 'retweet_count', 'reply_count', 'like_count', 'quote_count','tweet_device_code'
data = group_2_df[['num_words', 'num_sentences','word_per_sent','freq_dict_words','freq_words_ex','lex_diversity','mimicry_by_length','mimicry_by_word','num__beginning_sent_capital', 'num_punct_sent', 'num_uppers_sent']]
labels = group_2_df['username_code']
numFeatures = 11

Index(['user_id', 'tweet', 'username', 'username_code', 'num_words',
       'num_sentences', 'word_per_sent', 'freq_dict_words', 'freq_words_ex',
       'lex_diversity', 'num__beginning_sent_capital', 'num_punct_sent',
       'num_uppers_sent', 'mimicry_by_length', 'mimicry_by_word'],
      dtype='object')


In [21]:
### FEATURE RANKING
### https://machinelearningmastery.com/feature-selection-machine-learning-python/
## ANOVA F-value

# feature extraction
test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(data, labels)
# summarize scores
np.set_printoptions(precision=3)
print("List of Features and the scores\n")

print(tabulate([fit.scores_], headers=data.columns))
print("\n\n\n")

features = fit.transform(data)
# summarize selected features
print(tabulate(features[0:5,:], headers = data.columns))
print("The number of row, columns ",data.shape)

List of Features and the scores

  num_words    num_sentences    word_per_sent    freq_dict_words    freq_words_ex    lex_diversity    mimicry_by_length    mimicry_by_word    num__beginning_sent_capital    num_punct_sent    num_uppers_sent
-----------  ---------------  ---------------  -----------------  ---------------  ---------------  -------------------  -----------------  -----------------------------  ----------------  -----------------
    57.3864          40.9296            67.34            84.1164          1.80724          13.4121              2.59821             62.021                        47.5485            11.072            43.6816




  num_words    num_sentences    word_per_sent    freq_dict_words    freq_words_ex    lex_diversity    mimicry_by_length    mimicry_by_word    num__beginning_sent_capital    num_punct_sent    num_uppers_sent
-----------  ---------------  ---------------  -----------------  ---------------  ---------------  -------------------  --------------

In [22]:
# Open all files for writing everytime the program is ran from the beginning
filenames = ['output-group-2.csv','pca-output-group-2.csv']
#Header
header = ['Experiment-Number','Grouping-Number','Balance-Code','Classifier-Code','Split-Number','Accuracy','Total-F1','cm']
for outputFilename in filenames:
  # open the file using open() function
  with open(path+outputFilename, 'w', newline='') as f:
      writer = csv.writer(f)
      writer.writerow(header)

test_header = ['Experiment-Number','Grouping-Number','Balance-Code','Classifier-Code','Accuracy','Total-F1','cm']
for outputFilename in filenames:
  name = path+"test-"+outputFilename
  with open(name, 'w', newline='') as f:
      writer = csv.writer(f)
      writer.writerow(test_header)    

In [23]:
# Deal with data imbalance ofter stratified k fold split for cross validation
balanceTypes = ["None", "SMOTE", "RandomOver", "RandomUnder"]
analysisTypes = ["ANN", "SVM", "Logistic Regression", "Decision Tree", "Random Forest", "Naive Bayes"]
resultsList = []
testResultList = []

def perform_experiment(groupingCode, experimentNumber, data, labels, outputFilename):  
  #Split the overall dataset into  training and testing
  training_data, testing_data = train_test_split(data, test_size=0.1, random_state=34)
  training_labels = labels[training_data.index]
  testing_labels = labels[testing_data.index]

  print(f"No. of training examples: {training_data.shape[0]}")
  print(f"No. of testing examples: {testing_data.shape[0]}")

  models=[initNeuralNetModel(training_data, training_labels,numFeatures),
          SVC(kernel='rbf',gamma="auto"),
          LogisticRegression(random_state=34, C=1, multi_class='auto', solver='lbfgs',max_iter=100),
          DecisionTreeClassifier(criterion="gini", random_state=34, max_depth=5, min_samples_leaf=5),
          RandomForestClassifier(n_estimators=100), GaussianNB()]

  # Running classifiers for balance type = SMOTE
  balanceType = "SMOTE"
  resultsList = []
  testResultList = []
  # Using 5 splits for k-fold cross validation.
  skf = StratifiedKFold(n_splits=4,random_state=25, shuffle=True)
  split = -1
  for train, test in skf.split(training_data, training_labels):
      split = split + 1
      trainX = np.take(training_data, train, axis=0)
      trainY = np.take(training_labels, train, axis=0)
      testX = np.take(training_data, test, axis=0)
      testY = np.take(training_labels, test, axis=0)

      # deal with the imbalance on the training set alone (so duplication doesn't bleed across test/train split)
      trainX, trainY = balanceData(balanceType, trainX, trainY)

      for analysisType in analysisTypes:
          print("Doing analysis with {}".format(analysisType))
          if analysisType == "All":
              #print("Doing analysis with Linear Regression")
              #print("But it's not actually implemented...")
              preditions = testY + 1
          else:
              model = None
              if analysisType == "ANN":
                models[0] = doNeuralNetAnalysis(models[0], trainX, trainY, testX, testY, numFeatures)
                preds = models[0].predict(testX, verbose=0)
                predictions = preds.argmax(axis=1)
                #predictions = doNeuralNetAnalysis(trainX, trainY, testX, testY, numFeatures)
              elif analysisType == "SVM":
                  #svclassifier = SVC(kernel='linear')
                  #svclassifier = SVC(kernel='poly', degree=8)
                  #model = SVC(kernel='rbf', gamma="auto")
                  models[1].fit(trainX, trainY)
                  predictions = models[1].predict(testX)
                  #svclassifier = SVC(kernel='sigmoid')
              elif analysisType == "Logistic Regression":
                  #model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
                  models[2].fit(trainX, trainY)
                  predictions = models[2].predict(testX)   
              elif analysisType == "Decision Tree":
                  #model = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=3, min_samples_leaf=5)
                  models[3].fit(trainX, trainY)
                  predictions = models[3].predict(testX)
              elif analysisType == "Random Forest":
                  #model = RandomForestClassifier(n_estimators=100)
                  models[4].fit(trainX, trainY)
                  predictions = models[4].predict(testX)
              elif analysisType == "Naive Bayes":
                  #model = GaussianNB()
                  models[5].fit(trainX, trainY)
                  predictions = models[5].predict(testX)
              
              #scores = cross_val_score(model, data, labels, cv=5)
              print(classification_report(testY, predictions))
              cm = confusion_matrix(testY, predictions)
              print(cm)
              accuracy = accuracy_score(testY, predictions)
              print(f"Accuracy: {accuracy}")
              f1s = f1_score(testY, predictions, average=None)
              total_f1s = np.sum(f1s)
              print("Total f1 over classes: {}".format(total_f1s))
              balanceCode = getBalanceCode(balanceType)
              classifierCode = getClassifierCode(analysisType)
              turple = [experimentNumber, groupingCode, balanceCode, classifierCode, split, accuracy, total_f1s,cm.reshape([1,36]).tolist()]
              #print(turple)
              resultsList.append(turple)

  for i in range(len(models)):
      predictions = []
      print("Doing test analysis with {}".format(analysisTypes[i]))
      if analysisTypes[i] == "ANN":
        preds = models[0].predict(testing_data, verbose=0)
        predictions = preds.argmax(axis=1)
      else:
         predictions = models[i].predict(testing_data)

      #print(predictions)
      print(classification_report(testing_labels, predictions))
      cm = confusion_matrix(testing_labels, predictions)
      print(cm)
      accuracy = accuracy_score(testing_labels, predictions)
      print(f"Accuracy: {accuracy}")
      f1s = f1_score(testing_labels, predictions, average=None)
      total_f1s = np.sum(f1s)
      print("Total f1 over classes: {}".format(total_f1s))
      #featureCode = getFeatureCode(dataFiles, addBorders)
      balanceCode = getBalanceCode(balanceType)
      classifierCode = getClassifierCode(analysisTypes[i])
      turple = [experimentNumber, groupingCode, balanceCode, classifierCode, accuracy, total_f1s,cm.reshape([1,36]).tolist()]
      #print(turple)
      testResultList.append(turple)

  #print("****************************************************************")
  #print("EXPERIMENT VALIDATION RESULT")
  #print(tabulate(resultsList, headers = header))
  with open(path+outputFilename, 'a+', newline='') as f:
      writer = csv.writer(f)
      writer.writerows(resultsList)


  #print("****************************************************************")
  #print("EXPERIMENT TEST RESULT")
  #print(tabulate(resultsList, headers = header))
  name = path+"test-"+outputFilename
  with open(name, 'a+', newline='') as f:
      writer = csv.writer(f)
      writer.writerows(testResultList)     

perform_experiment('G-1', 1,data, labels,filenames[0])
perform_experiment('G-1', 2,data, labels,filenames[0])
perform_experiment('G-1', 3,data, labels,filenames[0])
perform_experiment('G-1', 4,data, labels,filenames[0])
perform_experiment('G-1', 5,data, labels,filenames[0])

No. of training examples: 1979
No. of testing examples: 220
Doing analysis with ANN
Doing analysis with fully connected neural network
[INFO] training network...
[INFO] evaluating network...
              precision    recall  f1-score   support

           0       0.35      0.15      0.21        96
           1       0.45      0.41      0.43        85
           2       0.21      0.52      0.30        84
           3       0.46      0.08      0.13        79
           4       0.38      0.58      0.46        72
           5       0.23      0.13      0.16        79

    accuracy                           0.31       495
   macro avg       0.35      0.31      0.28       495
weighted avg       0.35      0.31      0.28       495

[[14 15 41  0 14 12]
 [ 5 35 27  1 13  4]
 [ 5  4 44  4 17 10]
 [10  4 36  6 19  4]
 [ 2  8 14  2 42  4]
 [ 4 12 46  0  7 10]]
Accuracy: 0.30505050505050507
Total f1 over classes: 1.6862582164710018
Doing analysis with SVM
              precision    recall  f1-score

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.41      0.41      0.41        96
           1       0.53      0.59      0.56        85
           2       0.28      0.18      0.22        84
           3       0.32      0.30      0.31        79
           4       0.47      0.57      0.52        72
           5       0.36      0.42      0.39        79

    accuracy                           0.41       495
   macro avg       0.40      0.41      0.40       495
weighted avg       0.40      0.41      0.40       495

[[39 19 14 11  1 12]
 [15 50  5  3  4  8]
 [ 9  5 15 20 15 20]
 [11  9  9 24 15 11]
 [ 7  7  2  8 41  7]
 [14  5  8  8 11 33]]
Accuracy: 0.4080808080808081
Total f1 over classes: 2.400594675850099
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.21      0.19      0.20        96
           1       0.43      0.42      0.43        85
           2       0.44      0.14      0.22        84
           

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.36      0.42      0.39        97
           1       0.47      0.59      0.52        85
           2       0.24      0.14      0.18        83
           3       0.32      0.22      0.26        79
           4       0.46      0.53      0.49        72
           5       0.43      0.47      0.45        79

    accuracy                           0.39       495
   macro avg       0.38      0.39      0.38       495
weighted avg       0.38      0.39      0.38       495

[[41 14  9  7  8 18]
 [16 50  3  3  7  6]
 [22 16 12 14  9 10]
 [18 15 11 17 10  8]
 [ 4  8  7  7 38  8]
 [14  4  9  5 10 37]]
Accuracy: 0.3939393939393939
Total f1 over classes: 2.283595647387834
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.25      0.03      0.06        97
           1       0.43      0.31      0.36        85
           2       0.47      0.08      0.14        83
           

...

              precision    recall  f1-score   support

           0       0.38      0.45      0.41        97
           1       0.45      0.56      0.50        85
           2       0.28      0.18      0.22        83
           3       0.31      0.19      0.23        79
           4       0.47      0.50      0.48        72
           5       0.39      0.46      0.42        79

    accuracy                           0.39       495
   macro avg       0.38      0.39      0.38       495
weighted avg       0.38      0.39      0.38       495

[[44 14  6  5  8 20]
 [16 48  5  4  5  7]
 [21 17 15 14  9  7]
 [18 14 11 15  9 12]
 [ 6  8  5  7 36 10]
 [12  6 11  4 10 36]]
Accuracy: 0.39191919191919194
Total f1 over classes: 2.2704522966541605
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.25      0.03      0.06        97
           1       0.43      0.31      0.36        85
           2       0.47      0.08      0.14        83
         

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.34      0.33      0.34        97
           1       0.51      0.58      0.54        86
           2       0.18      0.16      0.17        83
           3       0.30      0.27      0.28        78
           4       0.49      0.57      0.53        72
           5       0.42      0.41      0.41        79

    accuracy                           0.38       495
   macro avg       0.37      0.39      0.38       495
weighted avg       0.37      0.38      0.38       495

[[32 18 14 16  7 10]
 [13 50  3  1 11  8]
 [18 12 13 16 10 14]
 [10  7 24 21  7  9]
 [ 7  7  7  7 41  3]
 [13  5 13  9  7 32]]
Accuracy: 0.38181818181818183
Total f1 over classes: 2.268707008999851
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.33      0.43      0.37        97
           1       0.49      0.47      0.48        86
           2       0.26      0.10      0.14        83
          

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.28      0.32      0.30        97
           1       0.50      0.49      0.50        85
           2       0.21      0.18      0.20        83
           3       0.28      0.23      0.25        79
           4       0.56      0.56      0.56        72
           5       0.44      0.53      0.48        78

    accuracy                           0.38       494
   macro avg       0.38      0.38      0.38       494
weighted avg       0.37      0.38      0.37       494

[[31 15 20 12  4 15]
 [12 42  7 11  4  9]
 [23  6 15 15  8 16]
 [19 11 16 18  8  7]
 [10  5  8  3 40  6]
 [15  5  4  5  8 41]]
Accuracy: 0.3785425101214575
Total f1 over classes: 2.2766847530537717
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.37      0.31      0.34        97
           1       0.40      0.44      0.42        85
           2       0.29      0.12      0.17        83
          

### Performing pca

In [24]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Scale data before applying PCA
#scaling=StandardScaler()
 
# Use fit and transform method
#scaling.fit(data)
#Scaled_data=scaling.transform(data)

# Set the n_components=3
principal=PCA(n_components=11)
principal.fit(data)
pca_data=principal.transform(data)

# summarize components
print("Explained Variance: %s" % principal.explained_variance_ratio_)
print("Shape of Transformed data: ",pca_data.shape)

pca_df = pd.DataFrame(pca_data, columns = ["pc"+str(pci) for pci in range(11)], index = labels.index)
print("PCA data frame has row,columns:", pca_df.shape)
perform_experiment('G-1', 1,pca_df, labels,filenames[1])
perform_experiment('G-1', 2,pca_df, labels,filenames[1])
perform_experiment('G-1', 3,pca_df, labels,filenames[1])
perform_experiment('G-1', 4,pca_df, labels,filenames[1])
perform_experiment('G-1', 5,pca_df, labels,filenames[1])

Explained Variance: [8.175e-01 7.990e-02 6.077e-02 1.751e-02 1.216e-02 8.528e-03 1.988e-03
 1.345e-03 2.146e-04 2.429e-05 1.267e-05]
Shape of Transformed data:  (2199, 11)
PCA data frame has row,columns: (2199, 11)
No. of training examples: 1979
No. of testing examples: 220
Doing analysis with ANN
Doing analysis with fully connected neural network
[INFO] training network...
[INFO] evaluating network...
              precision    recall  f1-score   support

           0       0.35      0.09      0.15        96
           1       0.44      0.56      0.49        85
           2       0.22      0.29      0.25        84
           3       0.31      0.18      0.23        79
           4       0.43      0.61      0.50        72
           5       0.41      0.52      0.46        79

    accuracy                           0.36       495
   macro avg       0.36      0.38      0.35       495
weighted avg       0.36      0.36      0.34       495

[[ 9 21 32  6  8 20]
 [ 3 48 21  2  6  5]
 [ 6  9 2

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.42      0.44      0.43        96
           1       0.43      0.47      0.45        85
           2       0.32      0.24      0.27        84
           3       0.36      0.30      0.33        79
           4       0.52      0.58      0.55        72
           5       0.37      0.43      0.40        79

    accuracy                           0.41       495
   macro avg       0.40      0.41      0.40       495
weighted avg       0.40      0.41      0.40       495

[[42 17 12  8  4 13]
 [19 40  6  9  3  8]
 [ 8 11 20 15 11 19]
 [11 11 13 24 12  8]
 [ 7  6  0  7 42 10]
 [13  7 12  4  9 34]]
Accuracy: 0.4080808080808081
Total f1 over classes: 2.428105223083046
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.33      0.74      0.45        96
           1       0.49      0.44      0.46        85
           2       0.36      0.15      0.22        84
           

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.38      0.44      0.41        97
           1       0.43      0.52      0.47        85
           2       0.28      0.18      0.22        83
           3       0.36      0.25      0.30        79
           4       0.48      0.50      0.49        72
           5       0.43      0.52      0.47        79

    accuracy                           0.40       495
   macro avg       0.39      0.40      0.39       495
weighted avg       0.39      0.40      0.39       495

[[43 16  8  6  7 17]
 [16 44  4  4  3 14]
 [19 15 15 16 12  6]
 [16 16 11 20  9  7]
 [ 7  8  4  6 36 11]
 [13  3 11  3  8 41]]
Accuracy: 0.402020202020202
Total f1 over classes: 2.3556342186022037
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.33      0.70      0.45        97
           1       0.61      0.41      0.49        85
           2       0.37      0.08      0.14        83
           

...

              precision    recall  f1-score   support

           0       0.31      0.37      0.34        97
           1       0.43      0.47      0.45        85
           2       0.26      0.14      0.18        83
           3       0.36      0.27      0.30        79
           4       0.48      0.49      0.48        72
           5       0.40      0.54      0.46        79

    accuracy                           0.38       495
   macro avg       0.37      0.38      0.37       495
weighted avg       0.37      0.38      0.37       495

[[36 17  8  7  9 20]
 [20 40  3  3  4 15]
 [22 13 12 15 11 10]
 [20 14  8 21  8  8]
 [ 7  6  6  7 35 11]
 [10  4 10  6  6 43]]
Accuracy: 0.37777777777777777
Total f1 over classes: 2.220637438600956
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.33      0.70      0.45        97
           1       0.61      0.41      0.49        85
           2       0.37      0.08      0.14        83
          

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.35      0.36      0.35        97
           1       0.50      0.56      0.53        86
           2       0.27      0.25      0.26        83
           3       0.27      0.21      0.23        78
           4       0.54      0.60      0.57        72
           5       0.39      0.39      0.39        79

    accuracy                           0.39       495
   macro avg       0.39      0.39      0.39       495
weighted avg       0.38      0.39      0.39       495

[[35 18 13 14  3 14]
 [14 48  6  4 10  4]
 [17 11 21 13 10 11]
 [14  7 23 16  5 13]
 [ 7  4  5  6 43  7]
 [14  8 11  7  8 31]]
Accuracy: 0.39191919191919194
Total f1 over classes: 2.331624728997453
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.32      0.67      0.43        97
           1       0.57      0.49      0.53        86
           2       0.29      0.13      0.18        83
          

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.37      0.34      0.35        97
           1       0.41      0.48      0.45        85
           2       0.32      0.24      0.28        83
           3       0.30      0.27      0.28        79
           4       0.62      0.64      0.63        72
           5       0.41      0.51      0.45        78

    accuracy                           0.41       494
   macro avg       0.40      0.41      0.41       494
weighted avg       0.40      0.41      0.40       494

[[33 20 14 13  1 16]
 [12 41  6  9  7 10]
 [13 12 20 14  5 19]
 [16  9 16 21  8  9]
 [ 6  7  3  6 46  4]
 [10 10  3  8  7 40]]
Accuracy: 0.4068825910931174
Total f1 over classes: 2.4391378601959737
Doing analysis with Naive Bayes
              precision    recall  f1-score   support

           0       0.33      0.64      0.44        97
           1       0.49      0.48      0.49        85
           2       0.42      0.17      0.24        83
          